In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/blood-cell-detection-dataset-master.zip

Archive:  /content/drive/MyDrive/blood-cell-detection-dataset-master.zip
721d642c0948d04e1995cee0bd172cdfedfad32e
   creating: blood-cell-detection-dataset-master/
 extracting: blood-cell-detection-dataset-master/.gitignore  
  inflating: blood-cell-detection-dataset-master/LICENSE  
  inflating: blood-cell-detection-dataset-master/README.md  
  inflating: blood-cell-detection-dataset-master/annotations.csv  
   creating: blood-cell-detection-dataset-master/images/
 extracting: blood-cell-detection-dataset-master/images/image-1.png  
 extracting: blood-cell-detection-dataset-master/images/image-10.png  
 extracting: blood-cell-detection-dataset-master/images/image-100.png  
 extracting: blood-cell-detection-dataset-master/images/image-101.png  
 extracting: blood-cell-detection-dataset-master/images/image-102.png  
 extracting: blood-cell-detection-dataset-master/images/image-103.png  
 extracting: blood-cell-detection-dataset-master/images/image-104.png  
 extracting: blood-cell-detec

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import glob
import os

img_path_all = glob.glob('/content/blood-cell-detection-dataset-master/images/*')
label_path_all = glob.glob('/content/blood-cell-detection-dataset-master/labels/*')

train_paths = img_path_all[:int(len(img_path_all)*0.8)]
val_paths = img_path_all[int(len(img_path_all)*0.8):int(len(img_path_all)*0.9)]
test_paths = img_path_all[int(len(img_path_all)*0.9):]

splits = {'train': train_paths, 'val': val_paths, 'test': test_paths}

for split in splits.keys():
    os.makedirs(f'/content/blood-cell-detection-dataset-master/{split}/images', exist_ok=True)
    os.makedirs(f'/content/blood-cell-detection-dataset-master/{split}/labels', exist_ok=True)

for split, files in splits.items():
    for file in files:
        os.rename(file, f'/content/blood-cell-detection-dataset-master/{split}/images/{os.path.basename(file)}')
        label_file = file.replace('images', 'labels').replace('.png', '.txt')
        os.rename(label_file, f'/content/blood-cell-detection-dataset-master/{split}/labels/{os.path.basename(label_file)}')

In [ ]:
import os
import glob
from PIL import Image

class_to_id = {
    'rbc': 0,
    'wbc': 1,
}

root = '/content/blood-cell-detection-dataset-master/'

for split in ['train','val','test']:
    img_dir     = os.path.join(root, split, 'images')
    lbl_dir     = os.path.join(root, split, 'labels')
    out_lbl_dir = os.path.join(root, split, 'labels_yolo')
    os.makedirs(out_lbl_dir, exist_ok=True)

    for lbl_path in glob.glob(os.path.join(lbl_dir, '*.txt')):
        base = os.path.splitext(os.path.basename(lbl_path))[0]
        # find matching image
        img_path = None
        for ext in ('.jpg','.jpeg','.png'):
            p = os.path.join(img_dir, base+ext)
            if os.path.isfile(p):
                img_path = p; break
        if img_path is None:
            print(f"[WARN] no image for {lbl_path}, skipping")
            continue

        W,H = Image.open(img_path).size

        yolo_lines = []
        with open(lbl_path) as f:
            for line in f:
                parts = line.strip().split()
                # need at least 8 columns: name,score?,?,? then xmin,ymin,xmax,ymax,...
                if len(parts) < 8:
                    continue
                cls_name = parts[0]
                if cls_name not in class_to_id:
                    continue
                # columns 4–7 are your bbox:
                x_min = float(parts[4])
                y_min = float(parts[5])
                x_max = float(parts[6])
                y_max = float(parts[7])

                # convert to YOLO format
                x_c = ((x_min + x_max)/2.0) / W
                y_c = ((y_min + y_max)/2.0) / H
                w   = (x_max - x_min)   / W
                h   = (y_max - y_min)   / H

                yolo_lines.append(f"{class_to_id[cls_name]} "
                                  f"{x_c:.6f} {y_c:.6f} {w:.6f} {h:.6f}")

        out_path = os.path.join(out_lbl_dir, base+'.txt')
        with open(out_path,'w') as fo:
            fo.write("\n".join(yolo_lines))
    print(f"✅ {split} done – labels_yolo ready")


✅ train done – labels_yolo ready
✅ val done – labels_yolo ready
✅ test done – labels_yolo ready


In [ ]:
%%writefile /content/blood-cell-detection-dataset-master/data.yaml
train: /content/blood-cell-detection-dataset-master/train/images
val: /content/blood-cell-detection-dataset-master/val/images
test: /content/blood-cell-detection-dataset-master/test/images

nc: 2
names: ['rbc', 'wbc']

Writing /content/blood-cell-detection-dataset-master/data.yaml


In [ ]:
!cat /content/blood-cell-detection-dataset-master/data.yaml

train: /content/blood-cell-detection-dataset-master/train/images
val: /content/blood-cell-detection-dataset-master/val/images
test: /content/blood-cell-detection-dataset-master/test/images

nc: 2
names: ['rbc', 'wbc']


In [ ]:
model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 69.6MB/s]


In [ ]:
results = model.train(data = '/content/blood-cell-detection-dataset-master/data.yaml', epochs = 100, imgsz = 256)

Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/blood-cell-detection-dataset-master/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru

train: Scanning /content/blood-cell-detection-dataset-master/train/labels.cache... 80 images, 0 backgrounds, 0 corrupt: 100%|██████████| 80/80 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 730.9±495.6 MB/s, size: 125.0 KB)



val: Scanning /content/blood-cell-detection-dataset-master/val/labels.cache... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<?, ?it/s]

Plotting labels to runs/detect/train2/labels.jpg... 


...

    100/100         0G     0.6614     0.4601     0.9076        288        256: 100%|██████████| 5/5 [00:13<00:00,  2.71s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.31it/s]

                   all         10        216      0.965      0.988       0.99      0.857



100 epochs completed in 0.422 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.68it/s]


                   all         10        216       0.96      0.976      0.986      0.862
                   rbc         10        205      0.947      0.951      0.977      0.816
                   wbc         10         11      0.973          1      0.995      0.907
Speed: 0.6ms preprocess, 48.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train2
